#Bem-vindo a aula 3 do Super módulo de API em Python

Na aula de hoje, veremos como modularizar o nosso projeto e como fazer conexão da nossa aplicação com um banco de dados.

##Modularização

Primeiro de tudo, vamos fazer a modularização do nosso projeto.

A modularização é uma das práticas mais importantes no desenvolvimento de software, especialmente em APIs backend. Ela consiste em dividir o sistema em partes menores, coesas e reutilizáveis, cada uma responsável por uma funcionalidade específica.

Neste "notebook", vamos explorar por que modularizar e como isso impacta a qualidade, manutenção e escalabilidade do projeto.
🔍 1. O Que é Modularização?

Modularizar significa:
✅ Separar o código em componentes lógicos (ex: autenticação, banco de dados, modelos).
✅ Definir interfaces claras entre os módulos.
✅ Evitar acoplamento excessivo (uma mudança em um módulo não quebra outros).

Exemplo de Estrutura Modular:
Copy

/projeto  
├── app.py                 # Ponto de entrada  
├── config/                # Configurações  
├── modules/  
│   ├── auth/              # Autenticação (JWT, OAuth)  
│   ├── users/             # Gestão de usuários  
│   └── products/          # Catálogo de produtos  
├── utils/                 # Funções auxiliares  
└── tests/                 # Testes automatizados  

🎯 2. Por Que Modularizar uma API?
📚 Legibilidade e Organização

    Código mais limpo: Cada arquivo/pasta tem um propósito claro.

    Facilita a navegação: Encontrar e editar funcionalidades é mais rápido.

    Documentação implícita: A estrutura do projeto revela sua lógica.

⚙️ Manutenção Simplificada

    Bugs são localizados mais rápido (ex: um erro na autenticação está em auth/).

    Atualizações são menos arriscadas (mudar um módulo não afeta outros).

🔄 Reutilização de Código

    Evita duplicação: Funções comuns (ex: conexão com banco) ficam em utils/database.py.

    Módulos podem ser compartilhados entre projetos (ex: um pacote auth-core para várias APIs).

🧪 Testabilidade

    Testes unitários focados: Cada módulo pode ser testado isoladamente.

    Mocks fáceis: Substitua dependências (ex: banco de dados) em testes.

🚀 Escalabilidade

    Novas features são adicionadas sem conflitos (ex: criar modules/payments/).

    Preparação para microserviços: Módulos podem virar serviços independentes no futuro.

👥 Trabalho em Equipe

    Desenvolvedores trabalham em paralelo (ex: um no auth/, outro no products/).

    Onboarding mais rápido: Novos membros entendem a estrutura facilmente.

⚠️ 3. Problemas sem Modularização

Um projeto não modularizado tende a:
❌ Virar um "God Object" (arquivo único com milhares de linhas).
❌ Dificultar testes e debugging.
❌ Causar conflitos frequentes no Git.
❌ Desencorajar boas práticas (ex: injeção de dependência).

Exemplo de Código Não Modularizado:
python
Copy

# app.py (TUDO misturado)  
def login():  
    # Lógica de autenticação + banco + JWT...  
def get_users():  
    # Consulta SQL + validação + formatação...  
def create_product():  
    # Regras de negócio + upload de arquivos...  

🛠️ 4. Como Começar a Modularizar?

    Identifique responsabilidades (ex: autenticação, banco de dados, modelos).

    Separe em pastas/arquivos (ex: modules/auth/routes.py, modules/auth/service.py).

    Use injeção de dependência (evite acoplamento direto entre módulos).

    Documente interfaces (ex: "auth.service deve receber um UserModel").

Ferramentas Recomendadas:

    SQLAlchemy (banco de dados organizado em modelos).

    Pydantic (validação de dados entre módulos).

📈 5. Benefícios a Longo Prazo
Antes	Depois
Código difícil de ler	Código autoexplicativo
Manutenção custosa	Atualizações rápidas
Time sobrecarregado	Devs produtivos
Bugs recorrentes	Erros localizáveis

Desse modo, vamos criar nossa aplicação modularizada no FastAPI, vamos criar algo semelhante a isso

app/
├── main.py

├── database/

│   └── session.py

├── models/

│   └── user.py

├── schemas/

│   └── user.py

├── crud/

│   └── user.py

└── routes/

|    └── user.py


Esse é um exemplo de uma aplicação de API muito eficiente, se tivessemos camadas de testes, poderíamos criar mais um módulo para testes e assim por diante.

##SQLAlchemy

Quando estamos trabalhando com banco de dados utilizamos uma linguagem chamada de SQL, essa linguagem é a linguagem oficial para banco de dados, por isso, para não termos a necessidade de nos aprofundarmos mais ainda na linguagem SQL e acabarmos perdendo a parte prática da aula, vamos falar sobre o SQLAlchemy.

SQLAlchemy é um ORM (Object-Relational Mapper) e uma biblioteca SQL toolkit para Python, que facilita a interação com bancos de dados relacionais de forma pythônica e eficiente.
📌 Principais Características
1. ORM (Mapeamento Objeto-Relacional)

Permite manipular tabelas do banco de dados como classes Python (modelos).

2. SQL Expression Language

Fornece uma maneira programática de escrever consultas SQL.

3. Suporte a Múltiplos Bancos de Dados

PostgreSQL, MySQL, SQLite, Oracle, Microsoft SQL Server, etc.

Trocar de banco é simples (muda apenas a connection string).

4. Sessões e Transações

Gerencia conexões e transações automaticamente.

5. Relacionamentos (Associações)

Define FKs, one-to-many, many-to-many, etc.

Na aula de hoje, vamos fazer um pequeno sistema com conexão ao banco de dados.

In [ ]:
# Arquivo database/session.py
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

DATABASE_URL = "sqlite:///./db.sqlite3"

engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(bind=engine, autocommit=False, autoflush=False)
Base = declarative_base()

Aqui, estamos criando uma sessão do nosso banco de dados, que estamos conectados ao banco de dados SQLite.

Agora, criando a nossa conexão de banco de dados, vamos criar um modelo para a nossa aplicação, um modelo é uma tabela.

In [ ]:
# models/user.py
from sqlalchemy import Column, Integer, String
from database.session import Base

class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, index=True, autoincrement=True)
    name = Column(String, nullable=False)
    email = Column(String, unique=True, index=True, nullable=False)

Após isso, é interessante que a gente crie uma camada para validação de dados, para que os dados sempre estejam tratados, antes de entrar na camada do banco de dados.

Essa camada, chamamos de schemas.

In [ ]:
from pydantic import BaseModel, EmailStr

class UserCreate(BaseModel):
    name: str
    email: EmailStr

    class Config:
        orm_mode = True

Após essa criação, desses arquivo com essa características, vamos criar uma camada para serviços, esses serviços normalmente englobam a lógica para por exeplo, criar um usuário, buscar usuário, atualizar usuário e etc.

In [ ]:
from fastapi import APIRouter, Depends, HTTPException
from sqlalchemy.orm import Session
from database.session import SessionLocal
from crud import user as crud_user
from schemas.user import UserCreate, UserOut
from typing import List

router = APIRouter()

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

@router.post("/users", response_model=UserOut)
def create_user(user: UserCreate, db: Session = Depends(get_db)):
    return crud_user.create_user(db, user)

@router.get("/users", response_model=List[UserOut])
def list_users(db: Session = Depends(get_db)):
    return crud_user.get_users(db)

@router.get("/users/{user_id}", response_model=UserOut)
def get_user(user_id: int, db: Session = Depends(get_db)):
    user = crud_user.get_user(db, user_id)
    if not user:
        raise HTTPException(status_code=404, detail="Usuário não encontrado")
    return user

@router.delete("/users/{user_id}")
def delete_user(user_id: int, db: Session = Depends(get_db)):
    success = crud_user.delete_user(db, user_id)
    if not success:
        raise HTTPException(status_code=404, detail="Usuário não encontrado")
    return {"ok": True}

Depois disso, não é interessante criarmos nossas rotas diretos na main.py, porque na main.py vamos ter apenas a responsabilidade única de executar o arquivo, então, os princípios de responsabilidades são essenciais para o desenvolvimento das aplicações.

In [ ]:
# routes/user.py
from fastapi import APIRouter, Depends
from sqlalchemy.orm import Session
from database.db import SessionLocal
from crud.user import create_user, get_users
from schemas.user import UserCreate, UserOut
from typing import List

router = APIRouter()

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

@router.post("/users", response_model=UserOut)
def create(user: UserCreate, db: Session = Depends(get_db)):
    return create_user(db, user)

@router.get("/users", response_model=List[UserOut])
def list_users(db: Session = Depends(get_db)):
    return get_users(db)


E agora, só falta o arquivo main.py, para que nossas rotas e nossa aplicação possa funcionar

In [ ]:
from fastapi import FastAPI
from routes import user

app = FastAPI()
app.include_router(user.router)